# Imports

In [959]:
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 100)

# Loading Index Files

In [872]:
df_eng_old_projects = pd.read_csv("G:\ESA_downloads\download_Bingjie_Mar262021\ESA_website_ENG_old_projects.csv", encoding='utf-8') # dtype fine
df_eng_new_projects = pd.read_csv("G:\ESA_downloads\phase_2_download_files\ESA_downloads\ESA_website_ENG_new_projects_fixed_encoding_errors.csv", encoding='utf-8') # dtypes fine

df_fra_old_projects = pd.read_csv("G:\ESA_downloads\download_Bingjie_May062021_fra\ESA_website_FRA_old_projects.csv", encoding='latin1') # dtype fine
df_fra_new_projects = pd.read_csv("G:\ESA_downloads\phase_2_download_files\ESA_downloads_fra\ESA_website_FRA_new_projects.csv", encoding='utf-8') # dtypes fine

df_eng_all_projects = pd.read_csv("G:\ESA_downloads\ESA_bundled_with_VCs\intermediate_files\ESA_website_ENG_fixed_encoding_errors.csv", encoding='utf-8') # dtypes fine

C:\Users\ThibJacq\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (52) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [873]:
# Standardizing dates to be in the format YYYY-MM-DD (best format / required format for GC)

df_eng_all_projects['Application Filing Date'] = pd.to_datetime(df_eng_all_projects['Application Filing Date'], format='%m/%d/%Y')
df_eng_all_projects['Application Filing Date'] = df_eng_all_projects['Application Filing Date'].dt.strftime('%Y-%m-%d')

df_eng_new_projects['Application Filing Date'] = pd.to_datetime(df_eng_new_projects['Application Filing Date'], format='%m/%d/%Y')
df_eng_new_projects['Application Filing Date'] = df_eng_new_projects['Application Filing Date'].dt.strftime('%Y-%m-%d')

df_fra_old_projects['Dépôt de la demande'] = pd.to_datetime(df_fra_old_projects['Dépôt de la demande'], format='%m/%d/%Y')
df_fra_old_projects['Dépôt de la demande'] = df_fra_old_projects['Dépôt de la demande'].dt.strftime('%Y-%m-%d')

df_fra_new_projects['Dépôt de la demande'] = pd.to_datetime(df_fra_new_projects['Dépôt de la demande'], format='%m/%d/%Y')
df_fra_new_projects['Dépôt de la demande'] = df_fra_new_projects['Dépôt de la demande'].dt.strftime('%Y-%m-%d')

In [874]:
df_eng_all_projects.drop(columns=['ID'], inplace=True)
df_eng_all_projects.drop(columns=['Data ID'], inplace=True)
df_eng_new_projects.drop(columns=['Data ID'], inplace=True)

In [875]:
df_eng_old_tables = df_eng_old_projects[df_eng_old_projects['Content Type'] == 'Table']

df_fra_old_tables = df_fra_old_projects[df_fra_old_projects['Type de contenu'] == 'Tableau']

In [876]:
print(len(df_eng_old_projects))
print(len(df_eng_new_projects))
print(len(df_fra_old_projects))
print(len(df_fra_new_projects))
print(len(df_eng_all_projects))
print(len(df_eng_old_projects) + len(df_eng_new_projects))
print(len(df_eng_old_tables))
print(len(df_fra_old_tables))

24285
644
24285
644
24929
24929
15151
15151


# Checking for duplicate rows

In [877]:
left_cols = ['Title', 'Document Number', 'PDF Page Number', 'Hearing order', 'File Name', 'Content Type']
len(df_eng_old_tables[left_cols].drop_duplicates(keep='first'))

15151

In [878]:
df_eng_old_tables[left_cols][df_eng_old_tables[left_cols].duplicated(keep='first')]

,Title,Document Number,PDF Page Number,Hearing order,File Name,Content Type


# Updating old and new index files with index file containing all projects and VCs

In [879]:
df_eng_all_projects.Title = df_eng_all_projects.Title.astype(str)
df_eng_new_projects.Title = df_eng_new_projects.Title.astype(str)
df_eng_old_projects['Regulatory Instrument(s)'] = df_eng_old_projects['Regulatory Instrument(s)'].astype(str)

In [880]:
df_eng_new_projects[df_eng_new_projects.Title.isnull()]

,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,ID,Project Download Path,Table Download Path,Good Quality,filename,Hearing Order


In [881]:
def merge_dfs_with_matching_specific_columns(left_df, right_df, left_on, right_on):
    """Merge two dataframes based on common columns and only keep rows where
    the values match.
    """
    merged_df = pd.merge(left_df, right_df, how='left', left_on=left_on, right_on=right_on, suffixes=('_x', ''))
    merged_df.drop(merged_df.filter(regex='_x$').columns, axis=1, inplace=True)
    return merged_df

left_cols = ['Title', 'Document Number', 'PDF Page Number']
right_cols = ['Title', 'Document Number', 'PDF Page Number']

df_eng_old_merged = merge_dfs_with_matching_specific_columns(df_eng_old_tables, df_eng_all_projects, left_on=left_cols, right_on=right_cols)
df_eng_new_merged = merge_dfs_with_matching_specific_columns(df_eng_new_projects, df_eng_all_projects, left_on=left_cols, right_on=right_cols)

In [882]:
len(df_eng_old_merged.drop_duplicates(subset=['Title', 'Document Number'])) # Checking to make sure number of rows is the same as the original

15151

In [883]:
# We had encoding issues which led to NaN values in the merged dataframe.
# The errors came from the table titles so I had to manually fix them and make the same in both dataframes.
# For future use, we will need to modify the code to handle encoding issues automatically.
# To do this, we will grab the index file from the database and use that file to label the VCs, then bundle the rows, 
# create French index by merging on ID column, and then concatenate the two dataframes. In other words, concatenate all eng projects in one file and fra in another.

assert len(df_eng_old_merged[df_eng_old_merged.ID.isna()]) == 0 # making sure the merge was successful and did not cause NaN values

In [884]:
assert len(df_eng_new_merged[df_eng_new_merged.ID.isna()]) == 0 # making sure the merge was successful and did not cause NaN values

In [885]:
df_eng_old_merged.shape

(15151, 52)

In [886]:
df_eng_new_merged.drop(columns=['filename', 'Hearing Order'], inplace=True)

# Creating the Merged French Index

In [887]:
df_fra_new_projects.drop(columns=['Data ID'], inplace=True)

In [888]:
left_cols = ['ID']
right_cols = ['ID']

In [889]:
# Checking to see that the columns we've chosen to identify the rows are the same for each item in the dataframes

mismatched_fr_items = []
mismatched_en_items = []

for right_col, left_col in zip(right_cols, left_cols):
    fr_test = df_fra_old_tables[left_col].unique()
    en_test = df_eng_old_tables[right_col].unique()
    
    for fr_item, en_item in zip(fr_test, en_test):
        if fr_item != en_item:
            print(fr_item, en_item)
            mismatched_fr_items.append(fr_item)
            mismatched_en_items.append(en_item)

print(len(mismatched_fr_items))

0


In [890]:
df_fra_old_merged = merge_dfs_with_matching_specific_columns(df_fra_old_tables, df_eng_old_merged, left_on=left_cols, right_on=right_cols)
df_fra_new_merged = merge_dfs_with_matching_specific_columns(df_fra_new_projects, df_eng_new_merged, left_on=left_cols, right_on=right_cols)

In [891]:
print(len(df_fra_old_merged))

15151


In [892]:
labels_list = ['Landscape, terrain, and weather', 
                'Soil',
                'Plants',
                'Water',
                'Fish',
                'Wetlands',
                'Wildlife',
                'Species at Risk',
                'Greenhouse gas emissions',
                'Air emissions',
                'Noise',
                'Electricity and electromagnetism',
                'Proximity to people',
                'Archaeological, paleontological, historical, and culturally significant sites and resources',
                'Human access to boats and waterways',
                'Indigenous land, water, and air use',
                'Impact to social and cultural well-being',
                'Impact to human health and viewscapes',
                'Social, cultural, economic infrastructure and services',
                'Economic Offsets and Impact',
                'Environmental Obligations',
                'Treaty and Indigenous Rights']

labels_list_fr = ['Paysage, terrain et conditions météorologiques',
                    'Sol',
                    'Plantes',
                    'Eau',
                    'Poissons',
                    'Milieux humides',
                    'Faune',
                    'Espèces en péril',
                    'Émissions de gaz à effet de serre',
                    'Émissions atmosphériques',
                    'Bruit',
                    'Électricité et électromagnétisme',
                    'Lieux habités à proximité',
                    'Sites et ressources archéologiques, paléontologiques, historiques ou importants sur le plan culturel',
                    'Activités à bord d’embarcation ou sur des voies navigables',
                    'Utilisation des terres, de l’eau et du plein air par les Autochtones',
                    'Incidences sur le bien-être social et culturel',
                    'Incidence sur la santé humaine et le panorama',
                    'Infrastructure et services sociaux, culturels et économiques',
                    'Compensations et répercussions économiques',
                    'Obligations en matière d’environnement',
                    'Droits ancestraux et issus de traités']

rename_eng_vcs_to_fra = {i:j for i,j in zip(labels_list, labels_list_fr)}

In [893]:
df_fra_new_merged.columns.difference(df_fra_new_projects.columns)

Index(['Air emissions', 'Application Filing Date', 'Application Name', 'Application Short Name', 'Application Type (NEB Act)', 'Application URL', 'Archaeological, paleontological, historical, and culturally significant sites and resources', 'Commodity', 'Company Name', 'Consultant Name', 'Content Type', 'Decision URL', 'Document Number', 'ESA Folder URL', 'ESA Section(s)', 'ESA Section(s) Index', 'ESA Section(s) Topics', 'Economic Offsets and Impact', 'Electricity and electromagnetism', 'Environmental Obligations', 'File Name', 'Fish', 'Good Quality', 'Greenhouse gas emissions', 'Hearing order', 'Human access to boats and waterways', 'Impact to human health and viewscapes', 'Impact to social and cultural well-being', 'Indigenous land, water, and air use', 'Landscape, terrain, and weather', 'Noise', 'PDF Download URL', 'PDF Outline', 'PDF Page Count', 'PDF Page Number', 'PDF Size', 'Pipeline Location', 'Pipeline Status', 'Plants', 'Project Download Path', 'Proximity to people',
       '

In [894]:
eng_columns_to_drop = df_fra_new_merged.columns.difference(df_fra_new_projects.columns).tolist()
eng_columns_to_drop = [elem for elem in columns_to_drop if elem not in labels_list]

In [895]:
eng_columns_to_drop.remove('filename')
eng_columns_to_drop.remove('Hearing Order')

In [896]:
df_fra_old_merged.drop(columns=eng_columns_to_drop, inplace=True)
df_fra_new_merged.drop(columns=eng_columns_to_drop, inplace=True)

In [897]:
df_fra_old_merged.rename(columns=rename_eng_vcs_to_fra, inplace=True)
df_fra_new_merged.rename(columns=rename_eng_vcs_to_fra, inplace=True)

# Adding the figures back into the old projects

In [898]:
df_eng_old_figures = df_eng_old_projects[df_eng_old_projects['Content Type'] == 'Figure']
df_fra_old_figures = df_fra_old_projects[df_fra_old_projects['Type de contenu'] == 'Figure']

In [899]:
# df_fra_old_figures.to_csv("G:\ESA_downloads\phase_2_download_files\ESA_downloads_fra\df_fra_old_figures.csv", index=False, encoding='utf-8-sig')

In [900]:
print(df_fra_old_figures.columns), print(df_fra_old_projects.columns)

Index(['Titre', 'Type de contenu', 'Nom de la demande', 'Nom abrégé de la demande', 'Dépôt de la demande', 'Nom de la société', 'Produit de base', 'Nom de fichier', 'URL du dossier de l'ÉES', 'Numéro de document', 'URL de téléchargement PDF', 'Type de demande (Loi sur l'Office national de l'énergie)', 'Emplacement du pipeline', 'Ordonnance d'audience', 'Nom du consultant', 'État d'avancement', 'Instruments réglementaires', 'URL de la demande', 'URL de la décision', 'Sections de l'EES', 'Index des sections de l'ÉES', 'Sujets des sections de l'ÉES', 'Numéro de page PDF', 'Nombre de pages PDF', 'Taille PDF', 'Aperçu PDF', 'ID', 'Chemin d'accès pour télécharger le projet', 'Chemin d'accès pour télécharger le tableau', 'Bonne qualité'], dtype='object')
Index(['Titre', 'Type de contenu', 'Nom de la demande', 'Nom abrégé de la demande', 'Dépôt de la demande', 'Nom de la société', 'Produit de base', 'Nom de fichier', 'URL du dossier de l'ÉES', 'Numéro de document', 'URL de téléchargement PDF',

(None, None)

In [901]:
df_eng_old_final = df_eng_old_merged.append(df_eng_old_figures)
df_fra_old_final = df_fra_old_merged.append(df_fra_old_figures)

In [902]:
df_eng_final = df_eng_new_merged.append(df_eng_old_final)
df_fra_final = df_fra_new_merged.append(df_fra_old_final)

In [951]:
df_eng_final.columns

Index(['Title', 'Document Number', 'PDF Page Number', 'ID', 'Content Type', 'Application Name', 'Application Short Name', 'Application Filing Date', 'Company Name', 'Commodity', 'File Name', 'ESA Folder URL', 'PDF Download URL', 'Application Type (NEB Act)', 'Pipeline Location', 'Hearing order', 'Consultant Name', 'Pipeline Status', 'Regulatory Instrument(s)', 'Application URL', 'Decision URL', 'ESA Section(s)', 'ESA Section(s) Index', 'ESA Section(s) Topics', 'PDF Page Count', 'PDF Size', 'PDF Outline', 'Landscape, terrain, and weather', 'Soil', 'Plants', 'Water', 'Fish', 'Wetlands', 'Wildlife', 'Species at Risk', 'Greenhouse gas emissions', 'Air emissions', 'Noise', 'Electricity and electromagnetism', 'Proximity to people', 'Archaeological, paleontological, historical, and culturally significant sites and resources', 'Human access to boats and waterways', 'Indigenous land, water, and air use', 'Impact to social and cultural well-being', 'Impact to human health and viewscapes',
      

In [904]:
df_fra_old_figures.shape

(9134, 30)

In [905]:
df_eng_final.shape

(24929, 52)

In [906]:
df_fra_final.shape

(24929, 52)

In [917]:
df_eng_final.sort_values(by=['Content Type', 'Application Filing Date', 'Application Short Name', 'PDF Page Number'], ascending=[False, False, True, True], inplace=True)

In [918]:
df_fra_final.sort_values(by=['Type de contenu', 'Dépôt de la demande', 'Nom abrégé de la demande', 'Numéro de page PDF'], ascending=[False, False, True, True], inplace=True)

In [938]:
import numpy as np
df_eng_final[labels_list] = df_eng_final[labels_list].replace(np.nan, 0)
df_fra_final[labels_list_fr] = df_fra_final[labels_list_fr].replace(np.nan, 0)

In [942]:
df_eng_final[labels_list] = df_eng_final[labels_list].astype(int)
df_fra_final[labels_list_fr] = df_fra_final[labels_list_fr].astype(int)

In [952]:
sorted_eng_columns = ['Title', 'Content Type', 'Application Name', 'Application Short Name', 'Application Filing Date', 'Company Name', 'Commodity', 'File Name', 'ESA Folder URL', 'Document Number', 'PDF Download URL', 'Application Type (NEB Act)', 'Pipeline Location', 'Hearing order', 'Consultant Name', 'Pipeline Status', 'Regulatory Instrument(s)', 'Application URL', 'Decision URL', 'ESA Section(s)', 'ESA Section(s) Index', 'ESA Section(s) Topics', 'PDF Page Number', 'PDF Page Count', 'PDF Size', 'PDF Outline', 'ID', 'Project Download Path', 'Table Download Path', 'Good Quality']
sorted_eng_columns.extend(labels_list)

sorted_fra_columns = ['Titre', 'Type de contenu', 'Nom de la demande', 'Nom abrégé de la demande', 'Dépôt de la demande', 'Nom de la société', 'Produit de base', 'Nom de fichier', 'URL du dossier de l\'ÉES', 'Numéro de document', 'URL de téléchargement PDF', 'Type de demande (Loi sur l\'Office national de l\'énergie)', 'Emplacement du pipeline', 'Ordonnance d\'audience', 'Nom du consultant', 'État d\'avancement', 'Instruments réglementaires', 'URL de la demande', 'URL de la décision', 'Sections de l\'EES', 'Index des sections de l\'ÉES', 'Sujets des sections de l\'ÉES', 'Numéro de page PDF', 'Nombre de pages PDF', 'Taille PDF', 'Aperçu PDF', 'ID', 'Chemin d\'accès pour télécharger le projet', 'Chemin d\'accès pour télécharger le tableau', 'Bonne qualité']
sorted_fra_columns.extend(labels_list_fr)

In [955]:
df_eng_final = df_eng_final.reindex(sorted_eng_columns, axis=1)
df_fra_final = df_fra_final.reindex(sorted_fra_columns, axis=1)

In [960]:
df_eng_final.head(2)

,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,ID,Project Download Path,Table Download Path,Good Quality,"Landscape, terrain, and weather",Soil,Plants,Water,Fish,Wetlands,Wildlife,Species at Risk,Greenhouse gas emissions,Air emissions,Noise,Electricity and electromagnetism,Proximity to people,"Archaeological, paleontological, historical, and culturally significant sites and resources",Human access to boats and waterways,"Indigenous land, water, and air use",Impact to social and cultural well-being,Impact to human health and viewscapes,"Social, cultural, economic infrastructure and services",Economic Offsets and Impact,Environmental Obligations,Treaty and Indigenous Rights
361,Table 7.1 Wildlife Key Indicators,Table,Application for the NGTL West Path Delivery 2023 Project,West Path Delivery 2023,2020-10-22,NOVA Gas Transmission Ltd.,Gas,C09063-11 NGTL WPD 2023_ESA_Part 2_Sections 7 to 18 - A7J7C9,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3974306,A7J7C9,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4003703,Large Projects (over 40 km),Alberta,GH-002-2020,Stantec Consulting Ltd,Applied,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3968941,NaN,"7.0 ASSESSMENT OF EFFECTS ON WILDLIFE AND WILDLIFE HABITAT, 8.0 ASSESSMENT OF EFFECTS ON AQUATIC...",NaN,NaN,4,612.0,13.74,Yes,362,/projects/wstpthdlvr2023.zip,/tables/wstpthdlvr2023_table-7-1-wildlife-key-indicators_pt-1_pg-4_doc-num-A7J7C9.zip,True,0,0,2,1,2,3,55,49,0,0,0,0,3,1,0,20,8,1,2,0,11,20
372,"Table 7.2 Potential Effects, Effect Pathways and Measurable Parameters for Wildlife and Wildlife...",Table,Application for the NGTL West Path Delivery 2023 Project,West Path Delivery 2023,2020-10-22,NOVA Gas Transmission Ltd.,Gas,C09063-11 NGTL WPD 2023_ESA_Part 2_Sections 7 to 18 - A7J7C9,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3974306,A7J7C9,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4003703,Large Projects (over 40 km),Alberta,GH-002-2020,Stantec Consulting Ltd,Applied,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3968941,NaN,"7.0 ASSESSMENT OF EFFECTS ON WILDLIFE AND WILDLIFE HABITAT, 8.0 ASSESSMENT OF EFFECTS ON AQUATIC...",NaN,NaN,6,612.0,13.74,Yes,373,/projects/wstpthdlvr2023.zip,/tables/wstpthdlvr2023_table-7-2-potential-effects-effect-pathways-and-me_pt-1_pg-6_doc-num-A7J7...,True,2,0,3,0,0,0,32,10,0,0,0,0,5,2,0,5,2,4,6,1,0,5


In [961]:
df_fra_final.head(2)

,Titre,Type de contenu,Nom de la demande,Nom abrégé de la demande,Dépôt de la demande,Nom de la société,Produit de base,Nom de fichier,URL du dossier de l'ÉES,Numéro de document,URL de téléchargement PDF,Type de demande (Loi sur l'Office national de l'énergie),Emplacement du pipeline,Ordonnance d'audience,Nom du consultant,État d'avancement,Instruments réglementaires,URL de la demande,URL de la décision,Sections de l'EES,Index des sections de l'ÉES,Sujets des sections de l'ÉES,Numéro de page PDF,Nombre de pages PDF,Taille PDF,Aperçu PDF,ID,Chemin d'accès pour télécharger le projet,Chemin d'accès pour télécharger le tableau,Bonne qualité,"Paysage, terrain et conditions météorologiques",Sol,Plantes,Eau,Poissons,Milieux humides,Faune,Espèces en péril,Émissions de gaz à effet de serre,Émissions atmosphériques,Bruit,Électricité et électromagnétisme,Lieux habités à proximité,"Sites et ressources archéologiques, paléontologiques, historiques ou importants sur le plan culturel",Activités à bord d’embarcation ou sur des voies navigables,"Utilisation des terres, de l’eau et du plein air par les Autochtones",Incidences sur le bien-être social et culturel,Incidence sur la santé humaine et le panorama,"Infrastructure et services sociaux, culturels et économiques",Compensations et répercussions économiques,Obligations en matière d’environnement,Droits ancestraux et issus de traités
350,Tableau 7.1 Indicateurs clés sur la faune,Tableau,Demande visant le projet de livraison parcours ouest de NGTL en 2023,Livraison parcours ouest 2023,2020-10-22,NOVA Gas Transmission Ltd.,Gaz,C09063-11 NGTL WPD 2023_ESA_Part 2_Sections 7 to 18 - A7J7C9,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Afficher/3974306,A7J7C9,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4003703,Projets de grande envergure (plus de 40 km),Alberta,GH-002-2020,Stantec Consulting Ltd,Demande présentée,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Afficher/3968941,NaN,"7.0 ÉVALUATION DES EFFETS SUR LA FAUNE ET L'HABITAT FAUNIQUE, 8.0 ÉVALUATION DES EFFETS SUR LES ...",NaN,NaN,4,612.0,NaN,NaN,362,/projects/wstpthdlvr2023.zip,/tables/wstpthdlvr2023_tableau-7-1-indicateurs-clés-sur-la-faune_pt-1_pg-4_num-du-doc-A7J7C9.zip,True,0,0,2,1,2,3,55,49,0,0,0,0,3,1,0,20,8,1,2,0,11,20
361,"Tableau 7.2 Effets éventuels, séquences des effets et paramètres mesurables pour la faune et l'h...",Tableau,Demande visant le projet de livraison parcours ouest de NGTL en 2023,Livraison parcours ouest 2023,2020-10-22,NOVA Gas Transmission Ltd.,Gaz,C09063-11 NGTL WPD 2023_ESA_Part 2_Sections 7 to 18 - A7J7C9,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Afficher/3974306,A7J7C9,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4003703,Projets de grande envergure (plus de 40 km),Alberta,GH-002-2020,Stantec Consulting Ltd,Demande présentée,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Afficher/3968941,NaN,"7.0 ÉVALUATION DES EFFETS SUR LA FAUNE ET L'HABITAT FAUNIQUE, 8.0 ÉVALUATION DES EFFETS SUR LES ...",NaN,NaN,6,612.0,NaN,NaN,373,/projects/wstpthdlvr2023.zip,/tables/wstpthdlvr2023_tableau-7-2-effets-éventuels-séquences-des-effets-_pt-1_pg-6_num-du-doc-A...,True,2,0,3,0,0,0,32,10,0,0,0,0,5,2,0,5,2,4,6,1,0,5


In [962]:
import time
current_time = time.strftime('%Y-%m-%d_%H-%M-%S')
print(current_time)

2021-09-11_16-33-16


In [963]:
df_eng_final.to_csv(f"G:\ESA_downloads\ESA_bundled_with_VCs\\archive\ESA_website_ENG_{current_time}.csv", index=False)

In [964]:
df_fra_final.to_csv(f"G:\ESA_downloads\ESA_bundled_with_VCs\\archive\ESA_website_FRA_{current_time}.csv", index=False, encoding='utf-8-sig')

In [965]:
df_eng_final.to_csv("G:\ESA_downloads\ESA_bundled_with_VCs\ESA_website_ENG_latest.csv", index=False)

In [966]:
df_fra_final.to_csv("G:\ESA_downloads\ESA_bundled_with_VCs\ESA_website_FRA_latest.csv", index=False, encoding='utf-8-sig')